In [1]:
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import tensorflow_probability as tfp
import scipy.special

tfd = tfp.distributions
np.random.seed(0)
import pickle

from LSM_tensorflow import *

2024-02-07 04:17:13.496847: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 04:17:13.506972: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 04:17:13.738839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 04:17:13.738880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 04:17:13.748837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
S0 = 27479.363
sigma = .1797
_lambda = 11.9488

eta1 = 1/0.074
eta2 = 1/0.0608
p = .3316
r = 0.01
d = 0.2

n = 200_000 # sample size
m = 360 # sample length
T = 1 #years

LTV0 = 0.6
LTVH = 0.8
liquidation_penalty = 0.95

degree = 2
seed = 0
print(n, m, LTV0, LTV0/LTVH)

200000 360 0.6 0.7499999999999999


In [4]:
LTVH = 0.9

start = -0.8
stop  = 0.8
step  = .05
all_d_arr = np.arange(start, stop+step, step)

start = 0.1
stop  = 0.8
step  = .05
LTV0_arr = np.arange(start, stop+step, step)

start = 10
stop  = m
step  = 10
slice_id_arr = np.arange(start, stop+step, step)

print(LTVH, all_d_arr, LTV0_arr, slice_id_arr)

0.9 [-8.00000000e-01 -7.50000000e-01 -7.00000000e-01 -6.50000000e-01
 -6.00000000e-01 -5.50000000e-01 -5.00000000e-01 -4.50000000e-01
 -4.00000000e-01 -3.50000000e-01 -3.00000000e-01 -2.50000000e-01
 -2.00000000e-01 -1.50000000e-01 -1.00000000e-01 -5.00000000e-02
  6.66133815e-16  5.00000000e-02  1.00000000e-01  1.50000000e-01
  2.00000000e-01  2.50000000e-01  3.00000000e-01  3.50000000e-01
  4.00000000e-01  4.50000000e-01  5.00000000e-01  5.50000000e-01
  6.00000000e-01  6.50000000e-01  7.00000000e-01  7.50000000e-01
  8.00000000e-01] [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85] [ 10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170 180
 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360]


In [5]:
d_arr = all_d_arr

In [6]:
payoffs_results = []

In [9]:
_Kou_model_params = Kou_model_params(S0, sigma, _lambda, eta1, eta2, p, r, T)

for d in d_arr:
    _price_paths_simulation = price_paths_simulation(seed, n, m, _Kou_model_params, d)
    _contract_params =  contract_params(_price_paths_simulation.d, LTV0, LTVH, _Kou_model_params, liquidation_penalty)
    _price_paths_simulation.gen_price_paths()

    for LTV0 in LTV0_arr:
        _contract_params_job = contract_params(_price_paths_simulation.d, LTV0, LTVH, _Kou_model_params, _contract_params.liquidation_penalty)

        payoffs, healthy_mask = gen_payoffs_and_healthy_mask(_contract_params_job, _price_paths_simulation.price_t,
                                                            _price_paths_simulation.path_mins)
        
        _mean = tf.math.reduce_mean(payoffs, axis=0)
        _var = tf.math.reduce_variance(payoffs, axis=0)
        payoffs_results.append([d, LTV0, _mean, _var])
        print(d, LTV0)

Price paths (price_t) and running minimums (path_mins) are generated.
-0.8 0.1
-0.8 0.15000000000000002
-0.8 0.20000000000000004
-0.8 0.25000000000000006
-0.8 0.30000000000000004
-0.8 0.3500000000000001
-0.8 0.40000000000000013
-0.8 0.45000000000000007
-0.8 0.5000000000000001
-0.8 0.5500000000000002
-0.8 0.6000000000000002
-0.8 0.6500000000000001
-0.8 0.7000000000000002
-0.8 0.7500000000000002
-0.8 0.8000000000000002
-0.8 0.8500000000000002
Price paths (price_t) and running minimums (path_mins) are generated.
-0.75 0.1
-0.75 0.15000000000000002
-0.75 0.20000000000000004
-0.75 0.25000000000000006
-0.75 0.30000000000000004
-0.75 0.3500000000000001
-0.75 0.40000000000000013
-0.75 0.45000000000000007
-0.75 0.5000000000000001
-0.75 0.5500000000000002
-0.75 0.6000000000000002
-0.75 0.6500000000000001
-0.75 0.7000000000000002
-0.75 0.7500000000000002
-0.75 0.8000000000000002
-0.75 0.8500000000000002
Price paths (price_t) and running minimums (path_mins) are generated.
-0.7 0.1
-0.7 0.15000000

In [10]:
with open('results_Euro2.pickle', 'wb') as handle:
    pickle.dump(payoffs_results, handle)
    print('saved')

saved
